## Creating Study Data Set from Example LMS Data

In [1]:
import pandas as pd
import numpy as np
import itertools
import os
import re
import sys
import ast
from datetime import timedelta

In [2]:
pd.set_option('display.max_columns', None)

# Read in Data 

In [3]:
path = 'example_data'
filelist = []

for root, _, files in os.walk(path):
    for file in files:
        filelist.append(os.path.join(root, file))

for f in filelist:
    print(f)

example_data/submission_comments.csv
example_data/assignments.csv
example_data/discussion_entry.csv
example_data/course_section.csv
example_data/submissions.csv
example_data/survey_data.csv
example_data/enrollments.csv
example_data/.gitkeep
example_data/assignments_overrides.csv


In [4]:
d = {}
for f in filelist: 
    if ".csv" not in f:
        continue
    entry_name = re.sub(r'\.csv$', '', f.rsplit('/', 1)[-1])
    d[entry_name] = pd.read_csv(f, delimiter=',')

# Data Set and Variable Overview 


In [5]:
# Each data frame is stored in one dictionary
for key in sorted(d.keys()):
    print('Variables of ' + key)
    print(sorted(d[key].columns.tolist()))
    print('')

Variables of assignments
['asn_due_at', 'asn_unlock_at', 'assignment_id', 'course_id', 'grading_type', 'workflow_state']

Variables of assignments_overrides
['assignment_id', 'due_at', 'unlock_at', 'updated_at']

Variables of course_section
['canvas_course_global_id', 'course_subject_name_number', 'section_num']

Variables of discussion_entry
['course_id', 'created_at', 'depth', 'discussion_entry_id', 'message_length', 'parent_discussion_entry_id', 'user_id']

Variables of enrollments
['course_id', 'enrollment_role_type', 'enrollment_state', 'enrollment_updated_at', 'user_id']

Variables of submission_comments
['author_id', 'course_id', 'message_size_bytes', 'submission_id']

Variables of submissions
['assignment_id', 'course_id', 'submission_id', 'submitted_at', 'user_id']

Variables of survey_data
['anon', 'avg_gpa', 'avg_grade', 'avg_major_gpa', 'course_name_number', 'credit_hours', 'grade_std', 'major', 'me', 'me_diff', 'me_importance', 'me_manage', 'n_prereqs', 'n_satisfied_prereq

# Feature Engineering Variable Overview

This is an overview of the features used in this study created from LMS records. Where the meaning of the variables can not be derived from their name, additional explanations are give. In addition, important definitions (e.g., what an original forum post is), how missing values were coded, as well as the expected associations of each feature with different course load types (i.e. time load: tl, mental effort: me, and psychological stress load: ps) are provided.

As per default, variables are created through concatenating data from all available sections students enrolled in. This is done through functions iterating through each row in the survey data and looking up relevant data in the LMS records, then creating the following variables:

*Note: Missing Values are represented by 0 and controlled for by binary variables representing whether a specific course used the forum, submission comments, assignment features in Canvas **at all**, with a 1 representing not used and a 0 representing used. Data can also later be omitted by referring to those binary variables prior to modelling. Some variables where 0s might naturally arise (e.g., dropout rate) might still contain NAs which will be cleaned in the R analysis script.*

**Forum Posts**

* n_original_forum_posts (made by students) - TL
    * defined as depth 1 posts in LMS records
    * with dropout version
    * NA if there are no discussion entries. 
* original_post_avg_size_bytes (made by students) - TL, maybe ME
    * with dropout version
    * NA if there are no original posts by students, since we can not input 0 which would indicate small size.
* original_forum_posts_per_student - TL
    * with dropout version
    * NA if no students participated in forum (can not divide by 0). 
* ta_teacher_posts_per_student - PS
    * ignores dropout status of students. 
    * NA if no students participated in forum.
* ta_teacher_avg_reply_time_minutes - PS
    * with dropout version
    * NA if no appropriate pair of student teacher interaction is available.
* forum_reply_ratio (reply from either student^/TA/instructor) | ^including dropout students
    * since posts having a reply influences the probability of whether a post received another reply, the reply can come from either students, dropout student, or instructors
    * with dropout version
    * NA if no relevant student (e.g., dropout student) particpated in forum.

**Submissions and Submission Comments**

* submission_comments_avg_size_bytes - maybe ME
    * ignores user_role (i.e. student, instructor, ...) since these could not be matched to the relevant tables
    * NA if there are no submission comments for a course/section numbers combination.
* submission_comments_per_student - maybe ME
    * ignores user_role (i.e. student, instructor, ...) since these could not be matched to the relevant tables
    * NA if there were no submissions to assignments by students for a course/section numbers combination.
* percent_submissions_submission_comments - PS
    * ignores user_role (i.e. student, instructor, ...) since these could not be matched to the relevant tables
    * NA if there were no relevant submissions to assignments by students in the course (i.e. can be 0).

**Assignments**

* assignment_spread (in SDs, with due date of assignment as reference) - PS
    * subset of assignments which had due dates
    * takes both graded and non-graded assignments
    * NA if there were less than 2 assignments with due
* parallel_assignments - PS
    * subset of assignments which had due dates
    * given a grace period of 1 day/3 days/1 day if ungraded & 3 if graded, how many pairs of timeframes overlapped?
* n_course_assignments - TL
    * based on all assignments (with or without due, unlock date)
    * NA if there were no assignments (Canvas assignments were not used)
* n_graded_assignments - TL 
    * based on all assignments (with or without due, unlock date)
    * NA if there were no assignments (Canvas assignments were not used)
* graded_assignments_week - TL
    * subset of assignments which had due dates
    * can be 0
* max_graded_assignments_week (number of graded assignments during week with most assignments) - PS
    * subset of assignments which had due dates
    * can be 0
* avg_submission_time_to_deadline_minutes (time between submission and deadline averaged over all submissions across courses) - PS
    * with dropout version
    * subset of courses which had due dates
    * NA if either there were no assignments nor submissions
* early_assignment_availability_ratio, ratio of frequency with which assignments are available within first 2 weeks of semester - PS, maybe ME
    * subset of courses which had due dates **and** unlock dates
    * Ratio of assignments unlocked before **two weeks** after semester start
    * NA if no assignments were put into Canvas (i.e. functionality was not used)
* avg_diff_available_due_assignments_minutes, assignment mean timeframe between availability and due date - PS
    * subset of courses which had due dates **and** unlock dates
    * NA if no assignments were put into Canvas (i.e. functionality was not used)


**Misc**

* dropout rates of course within 4 equally-sized quarters of instruction time - Maybe ME
    * Based on dividing Spring 2021 semester ('2021-01-18 00:00:00.000', '2021-05-13 23:59:59.999') into 4 equally sized parts and counting how many students' last enrollment status changed to dropout during that timeframe for the **primary** section of the course.
    * If no enrollment records are available for course name + primary section, return NA


# Preprocessing

In [6]:
# A reference table for adding user_roles to user_ids in the other tables
# is created and joined to the relevant tables. In our actual records,
# there were duplicate combinations of user x course x role which are
# omitted here. This is not relevant for the example data we created.

# Preselect variables
temp = d['enrollments'][['course_id', 'user_id', 'enrollment_role_type']]

# Filter Students, Teachers, TAs
temp = temp[temp['enrollment_role_type'].isin(['StudentEnrollment', 'TeacherEnrollment', 'TaEnrollment'])]

# Make strings cleaner
temp['enrollment_role_type'] = temp['enrollment_role_type'].str.replace('Enrollment', '') 

# Sort data frame such that Teacher and TA enrollment appear first
temp = temp.sort_values(by='enrollment_role_type', ascending=False)

# Drop duplicates such that first unique combination with Teacher or TA is kept
temp = temp.drop_duplicates(subset=['course_id', 'user_id'], keep='first', inplace=False)

user_role_reference_table = temp

# Simplify variable name
user_role_reference_table = user_role_reference_table.rename({'enrollment_role_type': 'user_role'}, axis=1)

### Left join user role to table `discussion entry` and `submissions`, check that no duplicate rows emerge

In [ ]:
print('Discussion Entry Table')
print(d['discussion_entry'].shape)
d['discussion_entry'] = d['discussion_entry'].merge(user_role_reference_table, on=['course_id', 'user_id'], how='left')
print(d['discussion_entry'].shape)
print(round(sum(pd.isna(d['discussion_entry'].user_role)) / d['discussion_entry'].shape[0], 4) * 100, '% missing user roles')
print('')
print('Submissions Table')
print(d['submissions'].shape)
d['submissions'] = d['submissions'].merge(user_role_reference_table, on=['course_id', 'user_id'], how='left')
print(d['submissions'].shape)
print(round(sum(pd.isna(d['submissions'].user_role)) / d['submissions'].shape[0], 4) * 100, '% missing user roles')
print('')
print('Submission Comments Table')
print(d['submission_comments'].shape)
d['submission_comments'] = d['submission_comments'].rename({'author_id': 'user_id'}, axis=1) # fix user id var name
d['submission_comments']['user_id'] = d['submission_comments'].user_id.fillna(0).astype(int) # fix encoding for correct joining
d['submission_comments'] = d['submission_comments'].merge(user_role_reference_table, on=['course_id', 'user_id'], how='left')
print(d['submission_comments'].shape)
print(round(sum(pd.isna(d['submission_comments'].user_role)) / d['submission_comments'].shape[0], 4) * 100, '% missing user roles')

### Create and join variable whether student is dropout or not: 1: dropout, 0: normal, -1: not student

In [8]:
# Take most recent enrollment state of each user in each course
temp = d['enrollments'][['course_id', 'user_id', 'enrollment_updated_at', 'enrollment_state']]
temp = temp.sort_values(by=['enrollment_updated_at', 'course_id', 'user_id'], ascending=False)
temp = temp.drop_duplicates(subset=['course_id', 'user_id'], keep='first', inplace=False)
enrollment_status = temp

In [ ]:
# Join last updated status including timestamp to tables
print('Discussion Entry Table')
print(d['discussion_entry'].shape)
d['discussion_entry'] = d['discussion_entry'].merge(enrollment_status, on=['course_id', 'user_id'], how='left')
print(d['discussion_entry'].shape)
print('')
print('Submissions Table')
print(d['submissions'].shape)
d['submissions'] = d['submissions'].merge(enrollment_status, on=['course_id', 'user_id'], how='left')
print(d['submissions'].shape)
print('')
print('Submission Comments Table')
print(d['submission_comments'].shape)
d['submission_comments'] = d['submission_comments'].merge(enrollment_status, on=['course_id', 'user_id'], how='left')
print(d['submission_comments'].shape)

In [10]:
# Create relevant variables
# A: If user is student and last updated enrollment status is deleted, then assign dropout status
#    If user is not student, assign -1, if user is student and not a dropout, assign 0 (active or completed)
# B: If user is a dropout student, return last updated enrollment status as time of dropout, else assign NA

def student_dropout_conditions(row):
    if row['user_role'] != 'Student':
        return -1
    else:
        if row['enrollment_state'] in ['active', 'completed']:
            return 0
        elif row['enrollment_state'] == 'deleted':
            return 1
        else:
            return np.nan
        
def dropout_at_conditions(row):
    if row['is_student_dropout'] != 1:
        return np.nan
    else:
        return row['enrollment_updated_at']

d['discussion_entry']['is_student_dropout'] = d['discussion_entry'].apply(student_dropout_conditions, axis=1)
d['submissions']['is_student_dropout'] = d['submissions'].apply(student_dropout_conditions, axis=1)
d['submission_comments']['is_student_dropout'] = d['submission_comments'].apply(student_dropout_conditions, axis=1)

d['discussion_entry']['dropout_at'] = d['discussion_entry'].apply(dropout_at_conditions, axis=1)
d['submissions']['dropout_at'] = d['submissions'].apply(dropout_at_conditions, axis=1)
d['submission_comments']['dropout_at'] = d['submission_comments'].apply(dropout_at_conditions, axis=1)

## Assignments Subsets for Feature Creation

1. Sample all assignments which are published in assignments table
    * Overwrite to assignments table
    * Base number of assignments and related variables on these assignments
2. Filter all assignments with updated deadlines
    * Save to separate data frame (`assignments_with_due`)
    * Base parallel assignments and related variables on these variables
3. Filter all assignments that have an updated unlock date (we do not know exactly when assignments were available for students for the remaining assignments
    * Save to separate data frame (`assignments_with_due-unlock`)
    * Base assignments availability variables on these assignments


### Update due date in `assignment` table through `assignment_overrides` table

In [11]:
updated_due_dates = d['assignments_overrides'].sort_values(by=['updated_at'], ascending=False)  # sort by most recent
updated_due_dates = updated_due_dates.loc[updated_due_dates['due_at'].notnull(), ['assignment_id', 'due_at']]
updated_due_dates.assignment_id = updated_due_dates.assignment_id.fillna(0).astype(int)
updated_due_dates = updated_due_dates.drop_duplicates(subset = 'assignment_id', keep = 'first') # keep most recent

# Join most recent entries to main table
d['assignments'] = pd.merge(d['assignments'], updated_due_dates, on='assignment_id', how='left')

# Take most recent due_at if available, else take asn_due_at from original table
d['assignments']['due_at_correct'] = d['assignments'][['asn_due_at', 'due_at']].apply(lambda x: x['asn_due_at'] if pd.isnull(x['due_at']) else x['due_at'], axis=1)

# there was one erreneuous row with the year 4444
d['assignments'] = d['assignments'][d['assignments']['due_at_correct']!='4444-04-05 06:59:59']

d['assignments']['due_at_correct'] = pd.to_datetime(d['assignments']['due_at_correct'])

### Update unlock date in `assignment` table through `assignment_overrides` table

In [12]:
# Step 1

updated_unlock_dates = d['assignments_overrides'].sort_values(by=['updated_at'], ascending=False)
updated_unlock_dates = updated_unlock_dates.loc[updated_unlock_dates['unlock_at'].notnull(), ['assignment_id', 'unlock_at']]
updated_unlock_dates.assignment_id = updated_unlock_dates.assignment_id.fillna(0).astype(int)
updated_unlock_dates = updated_unlock_dates.drop_duplicates(subset = 'assignment_id', keep = 'first')

d['assignments'] = pd.merge(d['assignments'], updated_unlock_dates, on='assignment_id', how='left')
d['assignments']['unlock_at_updated'] = d['assignments'][['asn_unlock_at', 'unlock_at']].apply(lambda x: x['asn_unlock_at'] if pd.isnull(x['unlock_at']) else x['unlock_at'], axis=1)

# there was one erreneuous row with the year 3333
d['assignments'] = d['assignments'][d['assignments']['unlock_at_updated']!='3333-03-03 08:00:00']

d['assignments']['unlock_at_updated'] = pd.to_datetime(d['assignments']['unlock_at_updated'])

### Preselect assignments

* Workflow state must be published
* Assignments should have at least one submission by students
* Assigns should not have a due date before the beginning of the the Spring 2021 semester

In [13]:
# Published workflow state

print('% of assignments that had an active workflow state in assignments_000 table:',
    '%.2f' % (round(d['assignments'][d['assignments'].workflow_state == 'published'].shape[0]/
        d['assignments'].shape[0], 4)* 100), '%'
)

d['assignments'] = d['assignments'][d['assignments'].workflow_state == 'published']

% of assignments that had an active workflow state in assignments_000 table: 32.13 %


In [14]:
# Assignments with at least 1 submission by students

assignment_ids_with_submissions = set(d['submissions'][
                                (d['submissions'].user_role == 'Student') & 
                                (d['submissions'].assignment_id.isin(d['assignments'].assignment_id))]\
                                      .assignment_id)

n_assignments_with_submissions = len(assignment_ids_with_submissions)
n_assignments = len(pd.unique(d['assignments'].assignment_id))

d['assignments'] = d['assignments'][d['assignments'].assignment_id.isin(assignment_ids_with_submissions)]

print('We confirmed that', \
      '%.2f' % (round(n_assignments_with_submissions/n_assignments, 4)*100), \
      '% of published assignments had submissions by students')

We confirmed that 1.04 % of published assignments had submissions by students


In [15]:
# Filter out assignments with due dates before semester start Spring 2021

semester_start = pd.to_datetime('2021-01-18 00:00:00.000')

print(
    '%.2f' % (round(d['assignments'].loc[~(
    (d['assignments'].due_at_correct.notna()) &
    (d['assignments'].due_at_correct < semester_start)
    ),].shape[0] / d['assignments'].shape[0], 4)*100), 
    '% of published assignments with at least 1 submission had due dates on or after the start of the semester'
)

d['assignments'] = d['assignments'].loc[~(
    (d['assignments'].due_at_correct.notna()) &
    (d['assignments'].due_at_correct < semester_start)
),]


100.00 % of published assignments with at least 1 submission had due dates on or after the start of the semester


In [16]:
## Apply Filtering for Subset Creation and Print Omission Stats

print(
'%.2f' % 
(round(sum(d['assignments'].due_at_correct.isna())/len(d['assignments'].due_at_correct), 4) * 100),
    '% of published assignments did not have a due date (after updating most recent non-empty value from overrides table)'
)

0.00 % of published assignments did not have a due date (after updating most recent non-empty value from overrides table)


In [17]:
d['assignments_with_due'] = d['assignments'][d['assignments'].due_at_correct.notna()]

In [18]:
print(
'%.2f' % 
(round(sum(d['assignments_with_due'].unlock_at_updated.isna())/len(d['assignments_with_due'].unlock_at_updated), 4) * 100),
    '% of those assignments did not have an unlock date (after updating most recent non-empty value from overrides table)'    
)

0.00 % of those assignments did not have an unlock date (after updating most recent non-empty value from overrides table)


In [19]:
d['assignments_with_due-unlock'] = d['assignments_with_due'][d['assignments_with_due'].unlock_at_updated.notna()]

# Reading in and Cleaning Survey Data

In [20]:
d['survey_data'] # This is synthetic, randomized data

,anon,course_name_number,section_num,secondary_section_number,n_prereqs,n_satisfied_prereqs_2021_Spring,n_satisfied_prereqs_all_past_semesters,credit_hours,avg_grade,grade_std,percentage_of_non_letter_grades,percentage_of_pass_or_satisfactory_among_non_letter_grades,tl_importance,me_importance,ps_importance,major,avg_gpa,avg_major_gpa,tl1,tl2,tl_manage,me,me_manage,ps,ps_manage,tl1_diff,tl2_diff,me_diff,ps_diff
0,0,Political Science 103,3,"[2, 3]",5,0,2,4,2.484848,1.232323,0.969697,0.383838,3,2,4,Business Administration,3.939394,2.303030,5,3,4,4,2,4,2,-1,-4,-1,-4
1,0,Political Science 103,3,"[2, 3]",4,3,0,3,3.878788,1.060606,0.929293,0.474747,4,3,3,L&S Data Science,3.090909,3.696970,4,2,4,1,5,5,2,-2,-4,2,-2
2,0,Physics 112,3,"[3, 2]",0,5,5,3,3.121212,1.555556,0.636364,0.333333,3,2,1,L&S Data Science,1.454545,1.515152,6,5,3,5,2,2,3,0,-4,-3,-1
3,0,Sociology 1,2,"[1, 1]",1,2,3,4,2.575758,1.050505,0.777778,0.747475,3,2,1,L&S Data Science,3.636364,2.454545,5,5,5,5,2,1,4,-5,4,-4,2
4,0,Political Science 103,2,"[3, 1]",0,4,1,3,3.424242,1.161616,0.282828,0.141414,4,4,2,L&S Data Science,1.606061,1.363636,3,5,1,4,4,3,2,0,4,-4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,249,Sociology 1,2,"[3, 2]",5,0,0,2,3.969697,1.121212,0.202020,0.848485,2,5,2,L&S Data Science,3.848485,3.727273,2,3,3,5,2,2,2,-4,2,-1,4
1246,249,Physics 112,2,"[2, 3]",1,4,2,3,3.272727,1.808081,0.828283,0.040404,5,2,2,L&S Data Science,1.121212,1.969697,6,5,3,5,1,3,4,-2,1,1,4
1247,249,Sociology 1,2,"[1, 1]",2,0,5,1,3.333333,1.363636,0.464646,0.474747,4,5,5,L&S Data Science,2.666667,2.666667,3,1,1,3,3,1,1,-3,2,-2,0
1248,249,Political Science 103,3,"[3, 1]",5,3,4,1,1.181818,1.050505,0.232323,0.121212,5,4,5,Business Administration,1.090909,2.454545,3,3,3,5,4,1,2,-3,0,-2,4


In [21]:
# survey_course_reference will be later used for data set filtering during variable creation
survey_course_reference = d['survey_data'][['course_name_number', 'section_num', 'secondary_section_number']]

In [22]:
# A hand-coded list of ANON UNIVERSITY Courses representing
# they are STEM courses or not. Coding based on:
# https://www.ice.gov/sites/default/files/documents/stem-list.pdf

d_stem_courses = {
"American Studies 101": False,
"American Studies C172": False,
"Anthropology 115": False,
"Anthropology 121AC": False,
"Anthropology 141": False,
"Anthropology 160AC": False,
"Anthropology 3AC": False,
"Architecture 11B": False,  # only lists Naval Architecture and Marine Engineering.
"Architecture 170B": False,  # only lists Naval Architecture and Marine Engineering.
"Architecture 198BC": False,  # only lists Naval Architecture and Marine Engineering.
"Asian Am & Asn Diaspora Stds 121": False,
"Asian Am & Asn Diaspora Stds 132AC": False,
"Asian Am & Asn Diaspora Stds 171": False,
"Asian Am & Asn Diaspora Stds 20A": False,
"Astronomy 84": True,
"Astronomy C12": True,
"Bioengineering 100": True,
"Bioengineering 104": True,
"Bioengineering 11": True,
"Bioengineering 110": True,
"Bioengineering 153": True,
"Bioengineering 25": True,
"Bioengineering 98": True,
"Biology 1A": True,
"Biology 1AL": True,
"Biology 1B": True,
"Business Admin-Undergrad 10": False,   # list includes business, but only business statistics and not administration
"Business Admin-Undergrad 102B": False,   # list includes business, but only business statistics and not administration,
"Business Admin-Undergrad 103": False,   # list includes business, but only business statistics and not administration,
"Business Admin-Undergrad 105": False,   # list includes business, but only business statistics and not administration,
"Business Admin-Undergrad 106": False,   # list includes business, but only business statistics and not administration,
"Business Admin-Undergrad 131": False,   # list includes business, but only business statistics and not administration,
"Business Admin-Undergrad 135": False,   # list includes business, but only business statistics and not administration,
"Business Admin-Undergrad 141": False,   # list includes business, but only business statistics and not administration,
"Business Admin-Undergrad 147": False,   # list includes business, but only business statistics and not administration,
"Business Admin-Undergrad 169": False,   # list includes business, but only business statistics and not administration,
"Business Admin-Undergrad 192T": False,   # list includes business, but only business statistics and not administration,
"Business Admin-Undergrad 194": False,   # list includes business, but only business statistics and not administration,
"Business Admin-Undergrad 198": False,   # list includes business, but only business statistics and not administration,
"Celtic Studies R1B": False,
"Chemical Engineering 141": True,
"Chemical Engineering 150A": True,
"Chemical Engineering 98": True,
"Chemistry 12B": True,
"Chemistry 1A": True,
"Chemistry 1AL": True,
"Chemistry 1B": True,
"Chemistry 3A": True,
"Chemistry 3AL": True,
"Chemistry 3B": True,
"Chemistry 3BL": True,
"Chemistry 98": True,
"Chinese 10Y": False,
"Chinese 1A": False,
"Civil & Environmental Eng 105": True,
"Civil & Environmental Eng 107": True,
"Civil & Environmental Eng 11": True,
"Civil & Environmental Eng 113": True,
"Civil & Environmental Eng 123": True,
"Civil & Environmental Eng 155": True,
"Civil & Environmental Eng 166": True,
"Civil & Environmental Eng 175": True,
"Civil & Environmental Eng 198": True,
"Civil & Environmental Eng 199": True,
"Civil & Environmental Eng C88": True,
"Classics 10B": False,
"Classics 130E": False,
"Classics 28": False,
"Cognitive Science 1": True,
"Cognitive Science 131": True,
"Cognitive Science 190": True,
"College Writing Programs R1A": False,
"College Writing Programs R4B": False,
"Computer Science 10": True,
"Computer Science 161": True,
"Computer Science 162": True,
"Computer Science 170": True,
"Computer Science 188": True,
"Computer Science 189": True,
"Computer Science 194": True,
"Computer Science 195": True,
"Computer Science 197": True,
"Computer Science 198": True,
"Computer Science 370": True,
"Computer Science 47B": True,
"Computer Science 61A": True,
"Computer Science 61B": True,
"Computer Science 61C": True,
"Computer Science 70": True,
"Computer Science 88": True,
"Computer Science W182": True,
"Computer Science W186": True,
"Data Science, Undergraduate 198": True,   # data science is usually listed as 11.0401 Information Science/Studies.
"Data Science, Undergraduate C100": True,
"Data Science, Undergraduate C104": True,
"Data Science, Undergraduate C8": True,
"Demography C175": False,
"Design Innovation 10": False,
"Design Innovation 15": False,
"Design Innovation 198": False,
"Design Innovation 98": False,
"Dutch 171AC": False,
"Earth & Planetary Science C12": True,
"Economics 1": False,     # list only lists econometrics/quantitative economics
"Economics 100A": False,
"Economics 100B": False,
"Economics 101A": False,
"Economics 115": False,
"Economics 157": False,
"Economics 172": False,
"Education 130": False,
"Education 197": False,
"Electrical Eng & Computer Sci 126": True,
"Electrical Eng & Computer Sci 127": True,
"Electrical Eng & Computer Sci 16A": True,
"Electrical Eng & Computer Sci 16B": True,
"Energy and Resources 98": True,
"Engineering 125": True,
"Engineering 26": True,
"Engineering 29": True,
"English 110": False,
"English 170": False,
"English 24": False,
"English 43B": False,
"English 45C": False,
"English R1B": False,
"Env Sci, Policy, & Mgmt 114": True,  # Environmental Science is Listed
"Env Sci, Policy, & Mgmt 131": True,
"Env Sci, Policy, & Mgmt 152": True,
"Env Sci, Policy, & Mgmt 40": True,
"Env Sci, Policy, & Mgmt 50AC": True,
"Env Sci, Policy, & Mgmt 98": True,
"Env Sci, Policy, & Mgmt 98BC": True,
"Env Sci, Policy, & Mgmt C167": True,
"Environ Econ & Policy C101": False,
"Environmental Design 100": False,
"Ethnic Studies 101A": False,
"Ethnic Studies 190": False,
"Ethnic Studies 197": False,
"Film 171": False,
"Film R1B": False,
"French 1": False,
"French 2": False,
"Gender & Womens Studies 100AC": False,
"Gender & Womens Studies 139": False,
"Geography 130": False, # only lists Geographic Information Science and Cartography.
"Geography 70AC": False,
"Global Poverty & Practice 105": False,
"Global Studies 110Q": False,
"Global Studies 173": False,
"Global Studies C10A": False,
"History 100M": False,
"History 109C": False,
"History 160": False,
"History 190": False,
"History 6B": False,
"History C139C": False,
"History R1B": False,
"History of Art 190F": False,
"Industrial Eng & Ops Rsch 135": True,
"Industrial Eng & Ops Rsch 162": True,
"Industrial Eng & Ops Rsch 165": True,
"Industrial Eng & Ops Rsch 166": True,
"Industrial Eng & Ops Rsch 170": True,
"Industrial Eng & Ops Rsch 173": True,
"Industrial Eng & Ops Rsch 185": True,
"Industrial Eng & Ops Rsch 186": True,
"Industrial Eng & Ops Rsch 190E": True,
"Industrial Eng & Ops Rsch 195": True,
"Industrial Eng & Ops Rsch 221": True,
"Industrial Eng & Ops Rsch 95": True,
"Information C265": True, # iSchool course on interface design
"Integrative Biology 169": True,
"Integrative Biology 192": True,
"Integrative Biology 198": True,
"Integrative Biology 77B": True,
"Integrative Biology 84": True,
"Integrative Biology 98": True,
"Integrative Biology 98BC": True,
"Integrative Biology C32": True,
"Interdisciplinary Studies 100J": False, # "The Social Life of Computing", historical and ethnographic methods
"Italian Studies R5B": False,
"Korean 10B": False,
"Korean 112": False,
"LGBT Studies 145": False,
"Landscape Arch & Env Plan 1": False, # only lists Naval Architecture and Marine Engineering.
"Latin 100": False,
"Letters & Science 22": False,  # interdisciplinary studies
"Letters & Science 25": False,  # interdisciplinary studies
"Linguistics 100": False, # only lists Cognitive Psychology and Psycholinguistics.
"Linguistics 115": False,
"Linguistics 47": False,
"Linguistics C105": False,
"Materials Science & Eng 45": True,
"Mathematics 104": True,
"Mathematics 10B": True,
"Mathematics 110": True,
"Mathematics 124": True,
"Mathematics 128A": True,
"Mathematics 152": True,
"Mathematics 160": True,
"Mathematics 16B": True,
"Mathematics 1A": True,
"Mathematics 1B": True,
"Mathematics 53": True,
"Mathematics 1B": True,
"Mathematics 53": True,
"Mathematics 54": True,
"Mathematics 55": True,
"Mathematics 98": True,
"Mathematics 98BC": True,
"Mechanical Engineering 104": True,
"Mechanical Engineering 40": True,
"Mechanical Engineering C85": True,
"Media Studies 111": False,
"Media Studies 113": False,
"Military Affairs 180": False,  # must be applied military technology to be STE
"Molecular & Cell Biology 100B": True,
"Molecular & Cell Biology 102": True,
"Molecular & Cell Biology 140": True,
"Molecular & Cell Biology 140L": True,
"Molecular & Cell Biology 198": True,
"Molecular & Cell Biology 199": True,
"Molecular & Cell Biology 38": True,
"Molecular & Cell Biology 50": True,
"Molecular & Cell Biology 90E": True,
"Molecular & Cell Biology C61": True,
"Molecular & Cell Biology C95B": True,
"Music 128": False,
"Music 159": False,
"Music 168B": False,
"Music 168C": False,
"Music 168CS": False,
"Music 170": False,
"Music 20A": False,
"Music 25": False,
"Music 27": False,
"Music 45M": False,
"Music 52A": False,
"Music 52B": False,
"Music 53A": False,
"Music 53B": False,
"Music 80": False,
"Music R1B": False,
"Near Eastern Studies 10": False,
"Near Eastern Studies 18": False,
"Nuclear Engineering 155": True,
"Nuclear Engineering 162": True,
"Nutritional Science & Tox 10S": True,
"Nutritional Science & Tox 11": True,
"Nutritional Science & Tox 160": True,
"Nutritional Science & Tox 170": True,
"Nutritional Science & Tox 190": True,
"Nutritional Science & Tox 198": True,
"Nutritional Science & Tox 20": True,
"Philosophy 104": False,
"Philosophy 121": False,
"Philosophy 12A": False,
"Philosophy 135": False,                                                                                                                                                                                       
"Philosophy 161": False,
"Philosophy 25B": False,
"Philosophy 3": False,
"Physical Education 1": False,
"Physics 112": True,
"Physics 137A": True,
"Physics 137B": True,
"Physics 7A": True,
"Physics 7B": True,
"Physics 8A": True,
"Physics 8B": True,
"Physics C21": True,
"Plant & Microbial Biology 122": True,
"Plant & Microbial Biology 40": True,
"Plant & Microbial Biology C112L": True,
"Political Science 103": False,
"Political Science 111AC": False,
"Political Science 112C": False,
"Political Science 146A": False,
"Political Science 148A": False,
"Political Science 149E": False,
"Political Science 149P": False,
"Political Science 179": False,
"Political Science 197": False,
"Political Science 2": False,
"Psychology 1": True,
"Psychology 110": True,
"Psychology 114": True,
"Psychology 130": True,
"Psychology 135": True,
"Psychology 160": True,
"Psychology 167AC": True,
"Psychology 198": True,
"Psychology 290B": True,
"Psychology C116": True,
"Psychology W1": True,
"Public Health 126": False, # only Veterinary Preventive Medicine, Epidemiology, and Public Health and Health Engineering
"Public Health 142": False,
"Public Health 150E": False,
"Public Health 198": False,
"Public Health W250B": False,
"Public Policy 101": False,
"Public Policy 157": False,
"Public Policy 192AC": False,
"Public Policy 198": False,
"Public Policy C103": False,
"Rhetoric R1B": False,
"Scandinavian 106": False,
"Slavic Languages & Lit R5B": False,
"Social Welfare 112": False,
"Social Welfare 114": False,
"Sociology 1": False,
"Sociology 127": False,
"Sociology 140": False,
"Sociology 167": False,
"Sociology 198": False,
"Sociology 3AC": False,
"Southeast Asian 148": False,
"Southeast Asian R5B": False,
"Spanish 131": False,
"Spanish 135": False,
"Statistics 133": True,
"Statistics 134": True,
"Statistics 135": True,
"Statistics 150": True,
"Statistics 20": True,
"Statistics 33B": True,
"Statistics 88": True,
"Statistics 89A": True,
"Statistics C131A": True,
"Statistics C140": True,
"Theater Dance & Perf Stds 111": False,
"Theater Dance & Perf Stds 172": False,
"Theater Dance & Perf Stds 52AC": False,
"Theater Dance & Perf Stds R1B": False,
"UGIS-UG Interdisc Studies 192A": False,
"UGIS-UG Interdisc Studies 192B": False,
"UGIS-UG Interdisc Studies 192D": False,
"UGIS-UG Interdisc Studies 192E": False,
"UGIS-UG Interdisc Studies C122": False
}

d_stem_courses = pd.DataFrame.from_dict(d_stem_courses, orient='index')\
    .reset_index()\
    .rename({'index': 'course_name_number', 0: 'is_stem_course'}, axis=1)

d['survey_data'] = d['survey_data'].merge(d_stem_courses, on='course_name_number', how='left')

In [23]:
# Same as above, but with majors
d_stem_majors = {
"Anthropology": False,
"Applied Mathematics": True,
"Architecture": False, 
"Bioengineering": True,
"Business Administration": False,
"Chemical Engineering": True,
"Chemistry": True,
"Civil & Environmental Eng": True,
"Civil Engineering": True,
"Cognitive Science": True,
"Computer Science": True,
"Economics": False,
"Electrical Eng & Comp Sci": True,
"Engineering Physics": True,
"English": False,
"Environmental Sciences": True,
"Gender & Womens Studies": False,
"Global Studies": False,
"Industrial Eng & Ops Rsch": True,
"Integrative Biology": True,
"L&S Computer Science": True,
"L&S Data Science": True,
"L&S Public Health": False,
"L&S Social Welfare": False,
"Letters & Sci Undeclared": np.NaN,
"Linguistics": False,
"MCB-Biochem & Mol Biol": True,
"MCB-Cell & Dev Biology": True,
"MCB-Genetics": True,
"MCB-Neurobiology": True,
"Mathematics": True,
"Mechanical Engineering": True,
"Media Studies": False,
"Microbial Biology": True,
"Molecular & Cell Biology": True,
"Molecular Environ Biology": True,
"Music": False,
"Nut Sci-Physio & Metabol": True,
"Nutritional Sci-Dietetics": True,
"Nutritional Sci-Toxicology": True,
"Nutritional Science": True,
"Physics": True,
"Political Economy": False,
"Political Science": False,
"Psychology": True,
"Public Health": False,
"Sociology": False,
"Statistics": True
}

d_stem_majors = pd.DataFrame.from_dict(d_stem_majors, orient='index')\
    .reset_index()\
    .rename({'index': 'major', 0: 'is_stem_student'}, axis=1)

d['survey_data'] = d['survey_data'].merge(d_stem_majors, on='major', how='left')

# Creating Variables 

General procedure: Write functions that create the variable for a specific combination of course_name_number and section names, return the variable for each combination, append the resulting vector of variable values to the original data frame of sampled courses and sections.

### Add Course Variables from other Tables to Relevant Tables

In [24]:
# Join course_name_number and section_num to other tables
# Note: Duplicate entries might emerge during joining
canvas_courses = d['course_section'][
    ['canvas_course_global_id', 'course_subject_name_number', 'section_num']
]
canvas_courses.columns = ['course_id', 'course_name_number', 'section_num']

d['submissions'] = d['submissions'].merge(canvas_courses, on='course_id', how='left')
d['submission_comments'] = d['submission_comments'].merge(canvas_courses, on='course_id', how='left')
d['assignments'] = d['assignments'].merge(canvas_courses, on='course_id', how='left')
d['assignments_with_due'] = d['assignments_with_due'].merge(canvas_courses, on='course_id', how='left')
d['assignments_with_due-unlock'] = d['assignments_with_due-unlock'].merge(canvas_courses, on='course_id', how='left')
d['discussion_entry'] = d['discussion_entry'].merge(canvas_courses, on='course_id', how='left')

In [25]:
d['enrollments'] = d['enrollments'].merge(canvas_courses, on='course_id', how='left')

### How many students dropped out of the course in each 4 quarters of the Spring Semester (as a fraction of all originally enrolled students)

In [26]:
# The dates for the Spring 2021 semester are January 18, 2021 to May 13, 2021.
semester_start = pd.to_datetime('2021-01-18 00:00:00.000')
semester_end = pd.to_datetime('2021-05-13 23:59:59.999')
semester_quarter_limits = pd.date_range(semester_start, semester_end, periods=5)

In [27]:
# Take most recent enrollment state of each user in each course
temp = d['enrollments'][['course_name_number', 'section_num', 'user_id', 'enrollment_updated_at', 'enrollment_state', 'enrollment_role_type']]
temp = temp.sort_values(by=['enrollment_updated_at', 'course_name_number', 'section_num', 'user_id'], ascending=False)
temp = temp.drop_duplicates(subset=['course_name_number', 'section_num', 'user_id'], keep='first', inplace=False)

# Filter students
temp = temp[temp['enrollment_role_type'] == 'StudentEnrollment']

# If students dropped out before start of Spring Semester, do not impute Spring Semester start date
# If students dropped out after end of Spring Semester, do not impute Spring Semester end date

# Rather, remove students from the calculation which dropped out outside of the semester
temp['enrollment_updated_at'] = pd.to_datetime(temp['enrollment_updated_at'])
temp = temp[(temp.enrollment_updated_at >= semester_start) & (temp.enrollment_updated_at <= semester_end)]

# Create "dropped out in quarter n" binary variables for each quarter
temp['dropped_out_q1'] = (temp['enrollment_state'] == 'deleted') & (
    semester_quarter_limits[0] <= temp['enrollment_updated_at']) & (
    temp['enrollment_updated_at'] <= semester_quarter_limits[1])
temp['dropped_out_q2'] = (temp['enrollment_state'] == 'deleted') & (
    semester_quarter_limits[1] <= temp['enrollment_updated_at']) & (
    temp['enrollment_updated_at'] <= semester_quarter_limits[2])
temp['dropped_out_q3'] = (temp['enrollment_state'] == 'deleted') & (
    semester_quarter_limits[2] <= temp['enrollment_updated_at']) & (
    temp['enrollment_updated_at'] <= semester_quarter_limits[3])
temp['dropped_out_q4'] = (temp['enrollment_state'] == 'deleted') & (
    semester_quarter_limits[3] <= temp['enrollment_updated_at']) & (
    temp['enrollment_updated_at'] <= semester_quarter_limits[4])

temp_dropout_reference = temp


In [28]:
# dropped_out_ratio_q{1,2,3,4}
def get_dropped_out_ratio(df, name1: str, section1: str, reference_var_quarter: str) -> float:
    """
    Calculates ratio of student dropout as a fraction of the total number of students that originally
    enrolled in the course. 
    Please parse dropped_out_q{1,2,3,4} to variable 'reference_var_quarter'.
    """
    temp = df[(df['course_name_number']==name1) & (df['section_num'] == section1)]
    if temp.shape[0] == 0: # if no enrollment records are available, return NA
        return np.nan
    else:
        return temp[temp[reference_var_quarter] == True].shape[0] / temp.shape[0]

In [29]:
dropout_ratio_q1 = []
for index, row in survey_course_reference.iterrows():
    dropout_ratio_q1.append(
        get_dropped_out_ratio(temp_dropout_reference, 
                               row['course_name_number'], 
                               row['section_num'], 
                               'dropped_out_q1')
    )

In [30]:
dropout_ratio_q2 = []
for index, row in survey_course_reference.iterrows():
    dropout_ratio_q2.append(
        get_dropped_out_ratio(temp_dropout_reference, 
                               row['course_name_number'], 
                               row['section_num'], 
                               'dropped_out_q2')
    )

In [31]:
dropout_ratio_q3 = []
for index, row in survey_course_reference.iterrows():
    dropout_ratio_q3.append(
        get_dropped_out_ratio(temp_dropout_reference, 
                               row['course_name_number'], 
                               row['section_num'], 
                               'dropped_out_q3')
    )

In [32]:
dropout_ratio_q4 = []
for index, row in survey_course_reference.iterrows():
    dropout_ratio_q4.append(
        get_dropped_out_ratio(temp_dropout_reference, 
                               row['course_name_number'], 
                               row['section_num'], 
                               'dropped_out_q4')
    )

### Submissions to Assignments

In [33]:
# submission_comments_avg_size_bytes
def get_submission_comments_avg_size_bytes(df, name1: str, section1: str, section2: list) -> float:
    temp = df[(df['course_name_number']==name1) & (df['section_num'].isin([section1] + section2))]
    if temp.shape[0] == 0: # if no forum data available on neither section, return NA
        return 0
    else:
        all_submission_comments = temp
        if all_submission_comments.shape[0] == 0: # if there are submission comments, we can not divide by 0
            return 0 
        else:
            return sum(all_submission_comments.message_size_bytes) / all_submission_comments.shape[0]

In [34]:
submission_comments_avg_size_bytes = []
for index, row in survey_course_reference.iterrows():
    submission_comments_avg_size_bytes.append(
        get_submission_comments_avg_size_bytes(d['submission_comments'], 
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   ast.literal_eval(row['secondary_section_number']))
    )

In [35]:
def get_submission_comments_per_student(df, name1: str, section1: str, section2: list) -> float:
    """
    Note: This function divides the number of submission comments made by TAs and teachers by the
    number of students in a course, regardless of the dropout status of students.
    """
    temp_comments = df['submission_comments'][(df['submission_comments']['course_name_number']==name1) & (df['submission_comments']['section_num'].isin([section1] + section2))]
    temp_submission = df['submissions'][(df['submissions']['course_name_number']==name1) & (df['submissions']['section_num'].isin([section1] + section2))]
    if temp_submission.shape[0] == 0: 
        return 0
    else:
        n_students_course = len(set(temp_submission[
            (temp_submission.user_role == 'Student')
        ].user_id))
        if n_students_course == 0: # if no students are enrolled, we can not divide by 0
            return 0
        else:
            all_submission_comments = temp_comments
            return all_submission_comments.shape[0]/n_students_course

In [36]:
submission_comments_per_student = []
for index, row in survey_course_reference.iterrows():
    submission_comments_per_student.append(
        get_submission_comments_per_student(d, 
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   ast.literal_eval(row['secondary_section_number']))
    )

In [37]:
def get_percent_submissions_submission_comments(df, name1: str, section1: str, section2: list) -> float:
    """
    Note: This function creates the intersection of submission IDs by students and submission comment
    parents IDs and returns the ratio of submission IDs by students which received submission comments.
    """
    temp_comments = df['submission_comments'][(df['submission_comments']['course_name_number']==name1) & (df['submission_comments']['section_num'].isin([section1] + section2))]
    temp_submission = df['submissions'][(df['submissions']['course_name_number']==name1) & (df['submissions']['section_num'].isin([section1] + section2))]
    if temp_submission.shape[0] == 0: 
        return np.nan
    else:
        # Get all submission IDs of submissions made by students that did not drop out
        all_orig_student_submission_ids = set(pd.unique(temp_submission.submission_id).tolist())

        # Get parent references from submission comments
        all_parent_submission_comment_ids = set(pd.unique(temp_comments.submission_id).tolist())
        
        if len(all_orig_student_submission_ids) == 0: # can not divide by 0
            return np.nan
        else:
            return len(all_orig_student_submission_ids & all_parent_submission_comment_ids) / len(all_orig_student_submission_ids)

In [38]:
percent_submissions_submission_comments = []
for index, row in survey_course_reference.iterrows():
    percent_submissions_submission_comments.append(
        get_percent_submissions_submission_comments(d,
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   ast.literal_eval(row['secondary_section_number']))
    )

### Assignments (number, spread, parallel, availability at beginning of semester)

In [39]:
def get_assignment_spread(df, name1: str, section1: str, section2: list) -> float:
    temp = df[(df.course_name_number==name1) & (df.section_num.isin([section1] + section2))]
    if temp.shape[0] in [0, 1]: # standard deviation requires 2+ data points
        return 0
    else:
        return temp.due_at_correct.astype(int).std()

In [40]:
assignment_spread = []
for index, row in survey_course_reference.iterrows():
    assignment_spread.append(
        get_assignment_spread(d['assignments_with_due'],
                               row['course_name_number'], 
                               row['section_num'], 
                               ast.literal_eval(row['secondary_section_number']))
    )

In [41]:
# parallel assignments
## Current approach which adds a time period in front of assignment deadlines and counts pair-wise overlap ##

# For all courses
# 1. Create a timeframe for each assignment from deadline-1day to deadline
# 2. Count number of timeframes that overlap in each course

def check_overlap(tuple1, tuple2, i, j):
    if i==j: # a timeframe will always overlap itself
        return False
    # this condition for overlap holds independent of which timeframe starts earlier
    elif tuple1[0] < tuple2[1] and tuple2[0] < tuple1[1]:
        return tuple(sorted([i, j])) # sort in order to filter out inverse later
    else:
        return False

In [42]:
def get_parallel_assingments(df, name1: str, section1: str, section2: list, grace_period_days: int) -> int:
    temp = df[(df.course_name_number==name1) & (df.section_num.isin([section1] + section2))].copy()
    if temp.shape[0] == 0:
        return np.nan
    else:
        temp['asn_frame_lag'] = d['assignments']['due_at_correct'] - timedelta(days=grace_period_days)
        temp['asn_frame'] = list(zip(temp.asn_frame_lag, temp.due_at_correct))
        out = []
        for i, timeframe1 in enumerate(temp.asn_frame):
            for j, timeframe2 in enumerate(temp.asn_frame):
                out.append(check_overlap(timeframe1, timeframe2, i, j))
        out = set([element for element in out if element != False and element is not None]) # casting to set drops inverse 
        return len(out)

In [43]:
parallel_assingments_1day = []
for index, row in survey_course_reference.iterrows():
    parallel_assingments_1day.append(
        get_parallel_assingments(d['assignments_with_due'],
                               row['course_name_number'], 
                               row['section_num'], 
                               ast.literal_eval(row['secondary_section_number']),
                               grace_period_days=1)
    )

In [44]:
parallel_assingments_3day = []
for index, row in survey_course_reference.iterrows():
    parallel_assingments_3day.append(
        get_parallel_assingments(d['assignments_with_due'],
                               row['course_name_number'], 
                               row['section_num'], 
                               ast.literal_eval(row['secondary_section_number']),
                               grace_period_days=3)
    )

In [45]:
# Flexible approach based on graded (3 days) or not graded (1 day)

def timeframe_conditions(row):
    res = 1 # 1 day, extend by factors and return value
    if row['grading_type'] in ['points', 'percent', 'letter_grade', 'gpa_scale']:
        res *= 3 
    return row['due_at_correct'] - timedelta(days=res)

def get_parallel_assingments_flexible(df, name1: str, section1: str, section2: list) -> int:
    temp = df[(df.course_name_number==name1) & (df.section_num.isin([section1] + section2))].copy()
    if temp.shape[0] == 0:
        return np.nan
    else:
        temp['asn_frame_start'] = temp.apply(timeframe_conditions, axis=1)
        temp['asn_frame'] = list(zip(temp.asn_frame_start, temp.due_at_correct))
        out = []
        for i, timeframe1 in enumerate(temp.asn_frame):
            for j, timeframe2 in enumerate(temp.asn_frame):
                out.append(check_overlap(timeframe1, timeframe2, i, j))
        out = set([element for element in out if element != False and element is not None]) # casting to set drops inverse 
        return len(out)

In [46]:
parallel_assingments_flexible = []
for index, row in survey_course_reference.iterrows():
    parallel_assingments_flexible.append(
        get_parallel_assingments_flexible(d['assignments_with_due'],
                               row['course_name_number'], 
                               row['section_num'], 
                               ast.literal_eval(row['secondary_section_number']))
    )

In [47]:
def get_n_course_assignments(df, name1: str, section1: str, section2: list, graded_only=False) -> int:
    temp = df[(df.course_name_number==name1) & (df.section_num.isin([section1] + section2))]
    if graded_only:
        temp = temp[temp['grading_type'].isin(['points', 'percent', 'letter_grade', 'gpa_scale'])]
    return 0 if temp.shape[0] == 0 else temp.shape[0]

In [48]:
n_course_assignments = []
for index, row in survey_course_reference.iterrows():
    n_course_assignments.append(
        get_n_course_assignments(d['assignments'],
                               row['course_name_number'], 
                               row['section_num'], 
                               ast.literal_eval(row['secondary_section_number']),
                               graded_only=False)
    )

In [49]:
n_course_assignments_graded = []
for index, row in survey_course_reference.iterrows():
    n_course_assignments_graded.append(
        get_n_course_assignments(d['assignments'],
                               row['course_name_number'], 
                               row['section_num'], 
                               ast.literal_eval(row['secondary_section_number']),
                               graded_only=True)
    )

In [50]:
semester_start = pd.to_datetime('2021-01-18 00:00:00.000')
semester_end = pd.to_datetime('2021-05-13 23:59:59.999')

weeks = []
while semester_start <= semester_end:
    weeks.append(semester_start)
    semester_start += timedelta(days=7)
    
week_start_dates = weeks[:-1]
week_end_dates = weeks[1:]

def get_graded_assignments_week(df, name1: str, section1: str, section2: list, metric='average', 
                               week_start_dates=week_start_dates, week_end_dates=week_end_dates):
    """
    Parse 'average' or 'max' as metric to get either the average number of graded assignments per week
    or the maximum number of assignments during the whole semester which was due in a single calendar week.
    """
    temp = df[(df.course_name_number==name1) & (df.section_num.isin([section1] + section2))]
    temp = temp[temp['grading_type'].isin(['points', 'percent', 'letter_grade', 'gpa_scale'])]
    
    assignments_due_per_week_list = []
    
    for week_start, week_end in zip(week_start_dates, week_end_dates):
        subset = temp[(temp['due_at_correct'] < week_end) & (week_start < temp['due_at_correct'])] # due this week only
        assignments_due_per_week_list.append(subset.shape[0])

    if metric == 'average':
        return sum(assignments_due_per_week_list)/len(assignments_due_per_week_list)
    elif metric == 'max':
        return max(assignments_due_per_week_list)
    else: raise ArgumentError('Please parse either "average" or "max" to argument "metric"')
    

In [51]:
graded_assignments_week_average = []
for index, row in survey_course_reference.iterrows():
    graded_assignments_week_average.append(
        get_graded_assignments_week(d['assignments_with_due'],
                               row['course_name_number'], 
                               row['section_num'], 
                               ast.literal_eval(row['secondary_section_number']),
                               metric='average')
    )

In [52]:
graded_assignments_week_max = []
for index, row in survey_course_reference.iterrows():
    graded_assignments_week_max.append(
        get_graded_assignments_week(d['assignments_with_due'],
                               row['course_name_number'], 
                               row['section_num'], 
                               ast.literal_eval(row['secondary_section_number']),
                               metric='max')
    )

In [53]:
def get_avg_submission_time_to_deadline_minutes(df, name1: str, section1: str, section2: list, dropout_status=0) -> float:
    temp_assignments = df['assignments_with_due'][(df['assignments_with_due'].course_name_number==name1) & (df['assignments_with_due']['section_num'].isin([section1] + section2))]
    temp_submission = df['submissions'][(df['submissions'].course_name_number==name1) & (df['submissions']['section_num'].isin([section1] + section2))]
    if temp_assignments.shape[0] == 0 or temp_submission.shape[0] == 0: 
        return 0
    else:
        # Join submission time of submissions table to respective assignment entry with due date
        join_this = temp_submission[
            (temp_submission.user_role == 'Student') &
            (temp_submission.is_student_dropout == dropout_status) 
        ][['assignment_id', 'submitted_at']]
        temp = pd.merge(temp_assignments[['course_id', 'assignment_id', 'due_at_correct']], 
                        join_this, on='assignment_id', how='left')

        # Where possible, create average timeframe difference in minutes
        temp['submitted_at'] = pd.to_datetime(temp['submitted_at']) 
        
        temp['submission_diff'] = temp['due_at_correct'] - temp['submitted_at']
        
        return temp.submission_diff.dt.total_seconds().mean()/60 

In [54]:
avg_submission_time_to_deadline_minutes = []
for index, row in survey_course_reference.iterrows():
    avg_submission_time_to_deadline_minutes.append(
        get_avg_submission_time_to_deadline_minutes(d,
                               row['course_name_number'], 
                               row['section_num'], 
                               ast.literal_eval(row['secondary_section_number']))
    )

In [55]:
semester_start_plus_two_weeks = pd.to_datetime('2021-01-18 00:00:00.000') + timedelta(days=14)

def get_early_assignment_availability_ratio(df, name1: str, section1: str, section2: list,
                                            semester_start_plus_two_weeks=semester_start_plus_two_weeks) -> float:
    temp = df[(df.course_name_number==name1) & (df.section_num.isin([section1] + section2))]
    if temp.shape[0] == 0:
        return np.nan
    else:
        return sum(temp['unlock_at_updated'] <= semester_start_plus_two_weeks) / temp.shape[0]

In [56]:
early_assignment_availability_ratio = []
for index, row in survey_course_reference.iterrows():
    early_assignment_availability_ratio.append(
        get_early_assignment_availability_ratio(d['assignments_with_due-unlock'],
                               row['course_name_number'], 
                               row['section_num'], 
                               ast.literal_eval(row['secondary_section_number']))
    )

In [57]:
def get_avg_diff_available_due_assignments(df, name1: str, section1: str, section2: list) -> float:
    temp = df[(df.course_name_number==name1) & (df.section_num.isin([section1] + section2))].copy()
    if temp.shape[0] == 0:
        return 0
    else:
        temp['diff_available_due'] = temp['due_at_correct'] - temp['unlock_at_updated']
        return temp.diff_available_due.dt.total_seconds().mean()/60 

In [58]:
avg_diff_available_due_assignments = []
for index, row in survey_course_reference.iterrows():
    avg_diff_available_due_assignments.append(
        get_avg_diff_available_due_assignments(d['assignments_with_due-unlock'],
                               row['course_name_number'], 
                               row['section_num'], 
                               ast.literal_eval(row['secondary_section_number']))
    )

### Forum posts and TA responsivity

In [59]:
def get_n_original_forum_posts(df, name1: str, section1: str, section2: str) -> int:
    temp = df[(df['course_name_number']==name1) & (df['section_num'].isin([section1, section2]))]
    if temp.shape[0] == 0: # if no forum data available on neither section, return NA
        return 0
    else:
        all_original_posts = temp[
            (temp.depth == 1) & 
            (temp.user_role == 'Student') &
            (temp.is_student_dropout == 0)]
        return all_original_posts.shape[0]

In [60]:
n_original_posts = []
for index, row in survey_course_reference.iterrows():
    n_original_posts.append(
        get_n_original_forum_posts(d['discussion_entry'], 
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   row['secondary_section_number'])
    )

In [61]:
def get_n_original_forum_posts_dropout(df, name1: str, section1: str, section2: str) -> int:
    temp = df[(df['course_name_number']==name1) & (df['section_num'].isin([section1, section2]))]
    if temp.shape[0] == 0: # if no forum data available on neither section, return NA
        return 0
    else:
        all_original_posts = temp[
            (temp.depth == 1) & 
            (temp.user_role == 'Student') &
            (temp.is_student_dropout == 1)]
        return all_original_posts.shape[0]

In [62]:
n_original_posts_dropout = []
for index, row in survey_course_reference.iterrows():
    n_original_posts_dropout.append(
        get_n_original_forum_posts_dropout(d['discussion_entry'], 
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   row['secondary_section_number'])
    )

In [63]:
def get_original_student_post_avg_size_bytes(df, name1: str, section1: str, section2: str) -> float:
    temp = df[(df['course_name_number']==name1) & (df['section_num'].isin([section1, section2]))]
    if temp.shape[0] == 0: # if no forum data available on neither section, return NA
        return 0
    else:
        all_original_posts = temp[
            (temp.depth == 1) & 
            (temp.user_role == 'Student') &
            (temp.is_student_dropout == 0)] 
        if all_original_posts.shape[0] == 0: # if there are no original posts by students, we can not divide by 0
            return 0
        else:
            return sum(all_original_posts.message_length) / all_original_posts.shape[0]

In [64]:
original_student_post_avg_size_bytes = []
for index, row in survey_course_reference.iterrows():
    original_student_post_avg_size_bytes.append(
        get_original_student_post_avg_size_bytes(d['discussion_entry'], 
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   row['secondary_section_number'])
    )

In [65]:
def get_original_student_post_avg_size_bytes_dropout(df, name1: str, section1: str, section2: str) -> float:
    temp = df[(df['course_name_number']==name1) & (df['section_num'].isin([section1, section2]))]
    if temp.shape[0] == 0: # if no forum data available on neither section, return NA
        return 0
    else:
        all_original_posts = temp[
            (temp.depth == 1) & 
            (temp.user_role == 'Student') &
            (temp.is_student_dropout == 1)] 
        if all_original_posts.shape[0] == 0: # if there are no original posts by students, we can not divide by 0
            return 0
        else:
            return sum(all_original_posts.message_length) / all_original_posts.shape[0]

In [66]:
original_student_post_avg_size_bytes_dropout = []
for index, row in survey_course_reference.iterrows():
    original_student_post_avg_size_bytes_dropout.append(
        get_original_student_post_avg_size_bytes_dropout(d['discussion_entry'], 
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   row['secondary_section_number'])
    )

In [67]:
def get_original_forum_posts_per_student(df, name1: str, section1: str, section2: list) -> float:
    """
    Note: This function divides the number of original posts made by students that did not drop out
    by the number of students that did not drop out and made at least one forum post (on all levels) in either 
    course section.
    """
    temp = df[(df['course_name_number']==name1) & (df['section_num'].isin([section1, section2]))]
    if temp.shape[0] == 0: # if no forum data available on neither section, return NA
        return 0
    else:
        n_students_course = len(set(temp[
            (temp.user_role == 'Student') &
            (temp.is_student_dropout == 0)
        ].user_id))
        if n_students_course == 0: # if no students participated in forum, we can not divide by 0
            return 0
        else :
            all_original_posts = temp[
                (temp.depth == 1) & 
                (temp.user_role == 'Student') &
                (temp.is_student_dropout == 0)]
            return all_original_posts.shape[0]/n_students_course

In [68]:
original_forum_posts_per_student = []
for index, row in survey_course_reference.iterrows():
    original_forum_posts_per_student.append(
        get_original_forum_posts_per_student(d['discussion_entry'], 
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   row['secondary_section_number'])
    )

In [69]:
def get_original_forum_posts_per_student_dropout(df, name1: str, section1: str, section2: list) -> float:
    """
    Note: This function divides the number of original posts made by students that did drop out
    by the number of students that did drop out and made at least one forum post (on all levels) in either 
    course section.
    """
    temp = df[(df['course_name_number']==name1) & (df['section_num'].isin([section1, section2]))]
    if temp.shape[0] == 0: # if no forum data available on neither section, return NA
        return 0
    else:
        n_students_course = len(set(temp[
            (temp.user_role == 'Student') &
            (temp.is_student_dropout == 1)
        ].user_id))
        if n_students_course == 0: # if no students participated in forum, we can not divide by 0
            return 0
        else :
            all_original_posts = temp[
                (temp.depth == 1) & 
                (temp.user_role == 'Student') &
                (temp.is_student_dropout == 1)]
            return all_original_posts.shape[0]/n_students_course

In [70]:
original_forum_posts_per_student_dropout = []
for index, row in survey_course_reference.iterrows():
    original_forum_posts_per_student_dropout.append(
        get_original_forum_posts_per_student_dropout(d['discussion_entry'], 
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   row['secondary_section_number'])
    )

In [71]:
def get_ta_teacher_posts_per_student(df, name1: str, section1: str, section2: str) -> float:
    """
    Note: This function divides the number of posts made by TAs or teachers
    by the number of students (regardless of dropout status)  that made at least one forum post 
    (on all levels) in either course section.
    """
    temp = df[(df['course_name_number']==name1) & (df['section_num'].isin([section1, section2]))]
    if temp.shape[0] == 0: # if no forum data available on neither section, return NA
        return 0
    else:
        n_students_course = len(set(temp[
            (temp.user_role == 'Student')
        ].user_id))
        if n_students_course == 0: # if no students participated in forum, we can not divide by 0
            return 0
        else :
            all_original_teacher_posts = temp[temp.user_role.isin(['Ta', 'Teacher'])]
            return all_original_teacher_posts.shape[0]/n_students_course

In [72]:
ta_teacher_posts_per_student = []
for index, row in survey_course_reference.iterrows():
    ta_teacher_posts_per_student.append(
        get_ta_teacher_posts_per_student(d['discussion_entry'], 
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   row['secondary_section_number'])
    )

In [73]:
def get_ta_teacher_reply_time(df, name1: str, section1: str, section2: list) -> float:
    """
    Note: This function returns the average reply time of TAs and teachers to posts by students 
    IF posts by students received a reply by a TA or teacher. Students are defined as students
    that did not drop out. A separate variable for students who dropped out will be created.
    """
    temp = df[(df['course_name_number']==name1) & (df['section_num'].isin([section1, section2]))]
    if temp.shape[0] == 0: # if no forum data available on neither section, return NA
        return np.nan
    else:
        reference_ids = temp[
            temp.depth != 1 & 
            temp.user_role.isin(['Ta', 'Teacher'])
        ][['course_id', 'parent_discussion_entry_id', 'discussion_entry_id', 'created_at']]
        
        # Get posting dates of parent IDs, if they are posts by students and by non-dropouts
        join_this = temp[
            (temp.user_role == 'Student') &
            (temp.is_student_dropout == 0)
        ][['discussion_entry_id', 'created_at']]
        
        # If parent to teacher reply is a post by  student non-dropout, join creation date of parent post
        diff = pd.merge(reference_ids, join_this, 
                 how='left', left_on=['parent_discussion_entry_id'], right_on=['discussion_entry_id'])
        
        # If parent is not such a post, an NA is joined, which is then omitted:
        diff = diff.dropna()
        
        if diff.shape[0] == 0:  # if no instance of reply time is available for neither section, then return NA
            return np.nan

        else:
            # Rename vars
            del diff['discussion_entry_id_y'] # redundant, this is the parent ID from the ta/teacher reply
            diff = diff.rename({'discussion_entry_id_x': 'discussion_entry_id', 'created_at_x': 'created_at_reply', 
                       'created_at_y': 'created_at_parent'}, axis=1)

            diff['created_at_reply'] = pd.to_datetime(diff['created_at_reply'])
            diff['created_at_parent'] = pd.to_datetime(diff['created_at_parent'])
            diff['reply_time_minutes'] = diff.apply(lambda x: (x['created_at_reply']-x['created_at_parent']), axis=1)
            
            return diff['reply_time_minutes'].dt.total_seconds().mean()/60 # convert to minutes

In [74]:
ta_teacher_reply_time = []
for index, row in survey_course_reference.iterrows():
    ta_teacher_reply_time.append(
        get_ta_teacher_reply_time(d['discussion_entry'], 
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   row['secondary_section_number'])
    )

In [75]:
def get_ta_teacher_reply_time_dropout(df, name1: str, section1: str, section2: list) -> float:
    """
    Note: This function returns the average reply time of TAs and teachers to posts by students 
    IF posts by students received a reply by a TA or teacher. Students are defined as students
    that dropped out.
    """
    temp = df[(df['course_name_number']==name1) & (df['section_num'].isin([section1, section2]))]
    if temp.shape[0] == 0: # if no forum data available on neither section, return NA
        return np.nan
    else:
        reference_ids = temp[
            temp.depth != 1 & 
            temp.user_role.isin(['Ta', 'Teacher'])
        ][['course_id', 'parent_discussion_entry_id', 'discussion_entry_id', 'created_at']]
        
        # Get posting dates of parent IDs, if they are posts by students and by non-dropouts
        join_this = temp[
            (temp.user_role == 'Student') &
            (temp.is_student_dropout == 1)
        ][['discussion_entry_id', 'created_at']]
        
        # If parent to teacher reply is a post by  student non-dropout, join creation date of parent post
        diff = pd.merge(reference_ids, join_this, 
                 how='left', left_on=['parent_discussion_entry_id'], right_on=['discussion_entry_id'])
        
        # If parent is not such a post, an NA is joined, which is then omitted:
        diff = diff.dropna()
        
        if diff.shape[0] == 0:  # if no instance of reply time is available for neither section, then return NA
            return np.nan

        else:
            # Rename vars
            del diff['discussion_entry_id_y'] # redundant, this is the parent ID from the ta/teacher reply
            diff = diff.rename({'discussion_entry_id_x': 'discussion_entry_id', 'created_at_x': 'created_at_reply', 
                       'created_at_y': 'created_at_parent'}, axis=1)

            diff['created_at_reply'] = pd.to_datetime(diff['created_at_reply'])
            diff['created_at_parent'] = pd.to_datetime(diff['created_at_parent'])
            diff['reply_time_minutes'] = diff.apply(lambda x: (x['created_at_reply']-x['created_at_parent']), axis=1)
            
            return diff['reply_time_minutes'].dt.total_seconds().mean()/60 # gets actually converted to mins here

In [76]:
ta_teacher_reply_time_dropout = []
for index, row in survey_course_reference.iterrows():
    ta_teacher_reply_time_dropout.append(
        get_ta_teacher_reply_time_dropout(d['discussion_entry'], 
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   row['secondary_section_number'])
    )

In [77]:
def get_reply_ratio(df, name1: str, section1: str, section2: str) -> float:
    """
    Note: This function returns the ratio of posts made by students that did not drop out
    that received a reply by any user (TA/teacher/students*) which also includes replies
    by students that dropped out because posts that already received a reply by these students
    are less likely to receive another reply.
    """
    temp = df[(df['course_name_number']==name1) & (df['section_num'].isin([section1, section2]))]
    if temp.shape[0] == 0: # if no forum data available on neither section, return NA
        return np.nan
    else:
        all_orig_student_post_ids = set(temp[
            (temp.user_role == 'Student') &
            (temp.is_student_dropout == 0)
        ].discussion_entry_id)
        
        if len(all_orig_student_post_ids) == 0: # if there are not posts made by students we can not divide by 0
            return np.nan
        else: 
            all_parent_post_ids = set(temp[
                (temp.user_role.isin(['Student', 'Ta', 'Teacher']))
            ].parent_discussion_entry_id)

            # The intersection of post IDs and parent post IDs represent those student posts that received replies
            return len(all_orig_student_post_ids & all_parent_post_ids)/len(all_orig_student_post_ids)

In [78]:
forum_reply_ratio = []
for index, row in survey_course_reference.iterrows():
    forum_reply_ratio.append(
        get_reply_ratio(d['discussion_entry'], 
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   row['secondary_section_number'])
    )

In [79]:
def get_reply_ratio_dropout(df, name1: str, section1: str, section2: str) -> float:
    """
    Note: This function returns the ratio of posts made by students that did drop out
    that received a reply by any user (TA/teacher/students*) which also includes replies
    by students that did not drop out because posts that already received a reply by these students
    are less likely to receive another reply.
    """
    temp = df[(df['course_name_number']==name1) & (df['section_num'].isin([section1, section2]))]
    if temp.shape[0] == 0: # if no forum data available on neither section, return NA
        return np.nan
    else:
        all_orig_student_post_ids = set(temp[
            (temp.user_role == 'Student') &
            (temp.is_student_dropout == 1)
        ].discussion_entry_id)
        
        if len(all_orig_student_post_ids) == 0: # if there are not posts made by students we can not divide by 0
            return np.nan
        else: 
            all_parent_post_ids = set(temp[
                (temp.depth != 1) & 
                (temp.user_role.isin(['Student', 'Ta', 'Teacher']))
            ].parent_discussion_entry_id)

            # The intersection of post IDs and parent post IDs represent those student posts that received replies
            return len(all_orig_student_post_ids & all_parent_post_ids)/len(all_orig_student_post_ids)

In [80]:
forum_reply_ratio_dropout = []
for index, row in survey_course_reference.iterrows():
    forum_reply_ratio_dropout.append(
        get_reply_ratio_dropout(d['discussion_entry'], 
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   row['secondary_section_number'])
    )

### Final Merging Process

In [81]:
# Add columns to survey data
dat = d['survey_data']

In [82]:
# Dropout Ratios
dat['dropout_ratio_q1'] = dropout_ratio_q1
dat['dropout_ratio_q2'] = dropout_ratio_q2
dat['dropout_ratio_q3'] = dropout_ratio_q3
dat['dropout_ratio_q4'] = dropout_ratio_q4

In [83]:
# Assignments, fix typos in object names
dat['assignment_spread'] = assignment_spread
dat['parallel_assignments_1day'] = parallel_assingments_1day
dat['parallel_assignments_3day'] = parallel_assingments_3day
dat['parallel_assignments_flexible'] = parallel_assingments_flexible
dat['n_course_assignments'] = n_course_assignments
dat['n_course_assignments_graded'] = n_course_assignments_graded
dat['graded_assignments_week_average'] = graded_assignments_week_average
dat['graded_assignments_week_max'] = graded_assignments_week_max
dat['avg_submission_time_to_deadline_minutes'] = avg_submission_time_to_deadline_minutes
dat['early_assignment_availability_ratio'] = early_assignment_availability_ratio
dat['avg_diff_available_due_assignments'] = avg_diff_available_due_assignments

# Submission Comments
dat['submission_comments_avg_size_bytes'] = submission_comments_avg_size_bytes
dat['submission_comments_per_student'] = submission_comments_per_student
dat['percent_submissions_submission_comments'] = percent_submissions_submission_comments

# Forum post quantity
dat['n_original_posts'] = n_original_posts
dat['n_original_posts_dropout'] = n_original_posts_dropout
dat['original_student_post_avg_size_bytes'] = original_student_post_avg_size_bytes
dat['original_student_post_avg_size_bytes_dropout'] = original_student_post_avg_size_bytes_dropout
dat['original_forum_posts_per_student'] = original_forum_posts_per_student
dat['original_forum_posts_per_student_dropout'] = original_forum_posts_per_student_dropout

# Forum post responsivity
dat['ta_teacher_posts_per_student'] = ta_teacher_posts_per_student
dat['ta_teacher_reply_time'] = ta_teacher_reply_time
dat['ta_teacher_reply_time_dropout'] = ta_teacher_reply_time_dropout
dat['forum_reply_ratio'] = forum_reply_ratio
dat['forum_reply_ratio_dropout'] = forum_reply_ratio_dropout

In [84]:
# Control variable
dat['holds_secondary_sections'] = dat['secondary_section_number'] == '[]'
dat.section_num = dat.section_num.astype(str)

In [85]:
# Binary control variables
dat['students_did_not_use_forum'] = dat['n_original_posts'] == 0
dat['teachers_ta_did_not_use_forum'] = dat['ta_teacher_posts_per_student'] == 0
dat['course_did_not_use_submission_comments'] = dat['submission_comments_per_student'] == 0
dat['course_did_not_use_assignments'] = dat['n_course_assignments'] == 0
dat['students_dropout_did_not_use_forum'] = dat['n_original_posts_dropout'] == 0
dat['course_did_not_use_forum'] = dat['students_did_not_use_forum'] & dat['teachers_ta_did_not_use_forum'] & dat['students_dropout_did_not_use_forum']

In [86]:
def get_n_course_assignments_deadline_unlock(df, name1: str, section1: str, section2: list, graded_only=False) -> int:
    """
    Returns the number of course assignments with deadlines or deadlines and unlock dates.
    Used for control variable creation.
    """
    temp = df[(df.course_name_number==name1) & (df.section_num.isin([section1] + section2))]
    return temp.shape[0]

In [87]:
n_course_assignments_deadline = []
for index, row in survey_course_reference.iterrows():
    n_course_assignments_deadline.append(
        get_n_course_assignments_deadline_unlock(d['assignments_with_due'],
                               row['course_name_number'], 
                               row['section_num'], 
                               ast.literal_eval(row['secondary_section_number']))
    )

In [88]:
dat['course_did_not_use_assignments_with_deadlines'] = pd.Series(n_course_assignments_deadline) == 0

In [89]:
n_course_assignments_deadline_unlock = []
for index, row in survey_course_reference.iterrows():
    n_course_assignments_deadline_unlock.append(
        get_n_course_assignments_deadline_unlock(d['assignments_with_due-unlock'],
                               row['course_name_number'], 
                               row['section_num'], 
                               ast.literal_eval(row['secondary_section_number']))
    )

In [90]:
dat['course_did_not_use_assignments_with_deadlines_unlock'] = pd.Series(n_course_assignments_deadline_unlock) == 0

In [91]:
def get_n_submissions_students(df, name1: str, section1: str, section2: str) -> float:
    """
    This function returns the total number of submissions by students regardless of dropout
    status to ascertain whether the submission feature was used in individual canvas courses
    """
    temp = df[(df['course_name_number']==name1) & (df['section_num'].isin([section1, section2]))]
    if temp.shape[0] == 0: # if no submission data available on neither section, return 0
        return 0
    else:
        all_submissions = temp[
            (temp.user_role == 'Student')
        ]
        return all_submissions.shape[0]

In [92]:
n_submissions_students = [] 
for index, row in survey_course_reference.iterrows():
    n_submissions_students.append(
        get_n_submissions_students(d['submissions'], 
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   row['secondary_section_number'])
    )

In [93]:
dat['course_did_not_use_submissions'] = pd.Series(n_submissions_students) == 0

In [94]:
def get_n_submissions_students_by_dropout(df, name1: str, section1: str, section2: str, dropout_status=0) -> float:
    """
    This function returns the total number of submissions by students by dropout
    status to ascertain whether the submission feature was used in individual canvas courses.
    """
    temp = df[(df['course_name_number']==name1) & (df['section_num'].isin([section1, section2]))]
    if temp.shape[0] == 0: # if no submission data available on neither section, return 0
        return 0
    else:
        all_submissions = temp[
            (temp.user_role == 'Student') &
            (temp.is_student_dropout == dropout_status)
        ]
        return all_submissions.shape[0]

In [95]:
n_submissions_students_non_dropout = [] 
for index, row in survey_course_reference.iterrows():
    n_submissions_students_non_dropout.append(
        get_n_submissions_students_by_dropout(d['submissions'], 
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   row['secondary_section_number'],
                                   dropout_status=0)
    )

In [96]:
dat['course_did_not_use_submissions_non_dropout'] = pd.Series(n_submissions_students_non_dropout) == 0

In [97]:
n_submissions_students_dropout = [] 
for index, row in survey_course_reference.iterrows():
    n_submissions_students_dropout.append(
        get_n_submissions_students_by_dropout(d['submissions'], 
                                   row['course_name_number'], 
                                   row['section_num'], 
                                   row['secondary_section_number'],
                                   dropout_status=1)
    )

In [98]:
dat['course_did_not_use_submissions_dropout'] = pd.Series(n_submissions_students_dropout) == 0

In [100]:
# Export
path = './aggregated_example_data/example_study_data.csv'
dat.to_csv(path, index=False) 